## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-22-22-54-50 +0000,nypost,Dr. Phil’s son’s NYPD documentary blocked by c...,https://nypost.com/2026/01/22/us-news/dr-phils...
1,2026-01-22-22-54-02 +0000,nypost,Trump says a ‘massive’ armada is heading towar...,https://nypost.com/2026/01/22/us-news/trump-sa...
2,2026-01-22-22-52-46 +0000,nyt,Live Updates: Zelensky Criticizes European All...,https://www.nytimes.com/live/2026/01/22/us/tru...
3,2026-01-22-22-52-40 +0000,nyt,Trump to Expand ‘Mexico City’ Abortion Rule to...,https://www.nytimes.com/2026/01/22/us/politics...
4,2026-01-22-22-52-33 +0000,nyt,House Rejects Measure to Bar Military Force in...,https://www.nytimes.com/2026/01/22/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,92
38,greenland,29
44,davos,22
75,ice,20
501,up,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
228,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...,171
268,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...,171
324,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,169
84,2026-01-22-21-19-43 +0000,bbc,"Board of Peace, Zelensky and Musk - What happe...",https://www.bbc.com/news/videos/c78vvvn4qnlo?a...,161
293,2026-01-22-10-34-23 +0000,nypost,Trump unveils ‘Board of Peace’ in Davos ceremo...,https://nypost.com/2026/01/22/us-news/trump-un...,160


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
228,171,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...
268,79,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...
118,66,2026-01-22-19-51-14 +0000,startribune,"Live: Activists arrested after church protest,...",https://www.startribune.com/ice-raids-minnesot...
46,56,2026-01-22-22-05-08 +0000,nypost,Davos has turned into an Art Basel party scene...,https://nypost.com/2026/01/22/business/world-e...
356,54,2026-01-22-02-17-39 +0000,cbc,Former Uvalde school police officer found not ...,https://www.cbc.ca/news/world/uvalde-police-sc...
347,53,2026-01-22-03-11-00 +0000,wsj,The Qatari-donated plane that is being overhau...,https://www.wsj.com/politics/national-security...
83,48,2026-01-22-21-22-31 +0000,nyt,"Zelensky Laces Into Europe, Saying It Must Ste...",https://www.nytimes.com/2026/01/22/world/europ...
33,45,2026-01-22-22-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
122,40,2026-01-22-19-48-00 +0000,wsj,"President Trump sued JPMorgan for $5 billion, ...",https://www.wsj.com/finance/president-trump-su...
103,37,2026-01-22-20-33-00 +0000,wsj,Judge Declines to Sign Off on Charges Against ...,https://www.wsj.com/us-news/law/judge-declines...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
